# Differences in Hierarchical Annotations

Let's investigate the SALAMI and SPAM annotations. For each hierarchical (2-level: upper and lower) segmentation, we will compute the following:

* NCE between upper and lower cases for each track between annotators
* Number of segments (upper and lower)
* Number of unique labels (upper and lower)
* Mean segment length (upper and lower)
* Number of segments per label (upper and lower)

In [53]:
from collections import namedtuple
import glob
import jams
import mir_eval
import numpy as np
import os
import pandas as pd

# Storing the intervals and labels of a segmentation in a named tuple
Segmentation = namedtuple('Segmentation', ['inters', 'labels'])

In [64]:
salami_jam_files = glob.glob("/home/uri/Projects/jams-data-salami/datasets/SALAMI/*.jams")
spam_jam_files = glob.glob("/home/uri/Projects/msaf-data/SPAM/references/*.jams")

In [66]:
def compute_metrics(upper, lower):
    """Computes the metrics for the given segmentations of a single annotator.
    
    Parameters
    ----------
    upper: Segmentation
        The upper segmentation of the annotator.
    lower: Segmentation
        The lower segmentation of the annotator.
        
    Returns
    -------
    res: dict
        Dictionary containing the following metrics:
        - NCE scores
        - number of segments
        - number of unique labels
        - mean segment length
        - number of segments per label
    """
    res = {}
    
    def _compute_layer_metrics(layer, prefix):
        """Computes single-layer metrics"""
        res["%snsegs" % prefix] = len(layer.labels)
        res["%snunique_labels" % prefix] = len(np.unique(layer.labels))
        res["%smean_seg_dur" % prefix] = np.mean([inter[1] - inter[0] 
                                                  for inter in layer.inters])
        res["%snsegs_per_label" % prefix] = \
            {label: len(np.where(np.asarray(layer.labels) == label)[0]) 
             for label in np.unique(layer.labels)}
    
    # Compute NCE
    try:
        res["S_o"], res["S_u"], res["S_f"] = \
            mir_eval.segment.nce(upper.inters, upper.labels, 
                                 lower.inters, lower.labels)
    except ValueError:
        res["S_o"], res["S_u"], res["S_f"] = (None, None, None)
    
    # Upper metrics
    _compute_layer_metrics(upper, "upper_")
    
    # Lower metrics
    _compute_layer_metrics(lower, "lower_")
    
    return res

def process_jam(jam_file):
    """Processes a given jam, obtaining all their annotators and having pairwise comparisons
    between upper and lower layers of segmentation."""
    jam = jams.load(jam_file)
    all_res = []
    for upper, lower in zip(jam.search(namespace="segment_salami_upper"),
                            jam.search(namespace="segment_salami_lower")):
        # Make sure we are dealing with the same annotator
        assert upper.annotation_metadata.annotator.name == lower.annotation_metadata.annotator.name

        # Get actual annotations
        upper_seg = Segmentation(*upper.data.to_interval_values())
        lower_seg = Segmentation(*lower.data.to_interval_values())

        # Compute all metrics
        res = compute_metrics(upper_seg, lower_seg)

        # Store additional info
        res["annotator_name"] = upper.annotation_metadata.annotator.name
        res["track_dur"] = jam.file_metadata.duration
        res["track_name"] = os.path.basename(jam_file)[:-5]
        all_res.append(res)
    return all_res

In [62]:
def compute_all_jams(jam_files):
    all_data = []
    for jam_file in jam_files:
        all_data += process_jam(jam_file)
    all_df = pd.DataFrame(all_data)
    all_df = all_df[["track_name", "track_dur", "annotator_name",
                     u'S_f', u'S_o', u'S_u', 
                     u'lower_mean_seg_dur',u'lower_nsegs', 
                     u'lower_nsegs_per_label', u'lower_nunique_labels',
                     u'upper_mean_seg_dur', u'upper_nsegs', 
                     u'upper_nsegs_per_label', u'upper_nunique_labels']]
    return all_df

In [71]:
# Compute all the salami metrics
salami_df = compute_all_jams(salami_jam_files)
salami_df.to_csv("../data/salami_annotator_metrics.tsv", sep="\t", index=False)

In [72]:
# Compute all the spam metrics
spam_df = compute_all_jams(spam_jam_files)
spam_df.to_csv("../data/spam_annotator_metrics.tsv", sep="\t", index=False)

In [75]:
spam_df.sort_values(by="S_f").head(5)

,track_name,track_dur,annotator_name,S_f,S_o,S_u,lower_mean_seg_dur,lower_nsegs,lower_nsegs_per_label,lower_nunique_labels,upper_mean_seg_dur,upper_nsegs,upper_nsegs_per_label,upper_nunique_labels
213,SALAMI_838,209.528163,Evan S. Johnson,0.139693,0.075091,1.000000,17.460680,12,"{u'a': 1, u'c': 1, u'b': 1, u'e': 1, u'd': 1, ...",12,104.764082,2,"{u'A': 1, u'SILENCE': 1}",2
27,SALAMI_458,240.065306,John Turner,0.172598,0.199353,0.152175,11.431681,21,"{u'a': 1, u'c': 5, u'b': 5, u'e': 4, u'd': 5, ...",6,60.016327,4,"{u'A': 1, u'C': 1, u'B': 1, u'ZZZZZ': 1}",4
107,SALAMI_1562,240.065306,John Turner,0.304138,0.248855,0.390997,5.595628,61,"{u'a': 13, u'c': 7, u'b': 8, u'e': 4, u'd': 8,...",12,48.761905,7,"{u'A': 1, u'C': 1, u'B': 1, u'D': 1, u'B'': 2,...",6
42,Cerulean_Miles_Davis_Quintet-Footprints,586.083265,John Turner,0.310648,0.308103,0.313235,8.140045,72,"{u'a': 23, u'c': 22, u'b': 22, u'a'': 4, u'zzz...",5,53.280297,11,"{u'A': 6, u'C': 1, u'B': 1, u'E': 1, u'D': 1, ...",6
157,SALAMI_478,251.062857,John Turner,0.361239,0.380641,0.343719,17.500106,30,"{u'a': 1, u'c': 6, u'b': 10, u'e': 4, u'd': 6,...",8,87.500529,6,"{u'A': 1, u'C': 1, u'B': 1, u'END': 1, u'D': 1...",6


In [96]:
salami_df.sort_values(by="S_f").head(5)

,track_name,track_dur,annotator_name,S_f,S_o,S_u,lower_mean_seg_dur,lower_nsegs,lower_nsegs_per_label,lower_nunique_labels,upper_mean_seg_dur,upper_nsegs,upper_nsegs_per_label,upper_nunique_labels
2232,53,275.190794,8,0.000000,0.106378,0.0,3.669211,75,"{u'a': 38, u'c': 12, u'b': 13, u'a'': 12}",4,21.168523,13,{u'B': 13},1
908,341,38.165601,7,0.000000,0.000000,0.0,6.360934,6,{u'b': 6},1,38.165601,1,{u'A': 1},1
1374,343,124.350272,8,0.000000,0.014677,0.0,8.882162,14,"{u'a': 8, u'a'': 6}",2,41.450091,3,{u'A': 3},1
30,32,197.448209,7,0.000000,0.030733,0.0,5.983279,33,"{u'a': 22, u'b': 11}",2,17.949837,11,{u'A': 11},1
1086,147,333.609796,4,0.058306,0.030028,1.0,9.016480,37,"{u'a': 2, u'a''''''''': 2, u'a'''': 2, u'b': 2...",20,83.402443,4,"{u'A': 3, u'Silence': 1}",2


In [91]:
salami_df[salami_df["track_name"] == "36"]

,track_name,track_dur,annotator_name,S_f,S_o,S_u,lower_mean_seg_dur,lower_nsegs,lower_nsegs_per_label,lower_nunique_labels,upper_mean_seg_dur,upper_nsegs,upper_nsegs_per_label,upper_nunique_labels
2043,36,180.680431,3,0.676081,0.510667,1.0,8.603830,21,"{u'a': 9, u'i': 1, u'b': 9, u'Silence': 2}",4,16.425494,11,"{u'A': 8, u'I': 1, u'Silence': 2}",3
2044,36,180.680431,4,0.549490,0.378826,1.0,10.036861,18,"{u'a': 6, u'b': 6, u'a'': 2, u'a''': 1, u'b'':...",6,18.066349,10,"{u'A': 8, u'B': 1, u'Silence': 1}",3


In [94]:
salami_df[salami_df["track_name"] == "220"]

,track_name,track_dur,annotator_name,S_f,S_o,S_u,lower_mean_seg_dur,lower_nsegs,lower_nsegs_per_label,lower_nunique_labels,upper_mean_seg_dur,upper_nsegs,upper_nsegs_per_label,upper_nunique_labels
720,220,123.992449,3,1.000000,1.000000,1.000000,6.525313,19,"{u'a': 11, u'i': 2, u'a'': 4, u'Silence': 2}",4,24.796190,5,"{u'A': 1, u'I': 1, u'Silence': 2, u'A'': 1}",4
721,220,123.992449,9,0.279739,0.371806,0.224217,9.537881,13,"{u'a': 6, u'b': 3, u'b'': 3, u'Silence': 1}",4,30.998113,4,"{u'A': 1, u'A''': 1, u'Silence': 1, u'A'': 1}",4


In [92]:
salami_df[salami_df["track_name"] == "923"]

,track_name,track_dur,annotator_name,S_f,S_o,S_u,lower_mean_seg_dur,lower_nsegs,lower_nsegs_per_label,lower_nunique_labels,upper_mean_seg_dur,upper_nsegs,upper_nsegs_per_label,upper_nunique_labels
1407,923,124.074172,7,0.687489,0.614351,0.780395,9.541169,13,"{u'a': 6, u'i': 1, u'b': 4, u'Silence': 2}",4,13.781688,9,"{u'A': 1, u'C': 4, u'B': 2, u'Silence': 2}",4
1408,923,124.074172,4,0.415577,0.262289,1.000000,11.279470,11,"{u'a': 2, u'c': 4, u'b': 4, u'Silence': 1}",4,62.037086,2,"{u'A': 1, u'Silence': 1}",2


In [93]:
salami_df[salami_df["track_name"] == "254"]

,track_name,track_dur,annotator_name,S_f,S_o,S_u,lower_mean_seg_dur,lower_nsegs,lower_nsegs_per_label,lower_nunique_labels,upper_mean_seg_dur,upper_nsegs,upper_nsegs_per_label,upper_nunique_labels
151,254,188.396122,7,0.953187,0.910561,1.0,10.466451,18,"{u'a': 8, u'c': 2, u'b': 2, u'e': 1, u'd': 2, ...",7,18.839612,10,"{u'A': 1, u'C': 1, u'B': 4, u'E': 1, u'D': 1, ...",6
152,254,188.396122,8,1.000000,1.000000,1.0,10.464943,18,"{u'a': 8, u'c': 2, u'b': 2, u'e': 1, u'd': 2, ...",7,17.124453,11,"{u'A': 4, u'C': 1, u'B': 1, u'E': 1, u'D': 1, ...",7
